In [1]:
import copy
import random

import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models, callbacks

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, X_test = X_train / 255, X_test / 255

X_train = X_train.reshape((60000, 28 ,28, 1))
X_test = X_test.reshape((10000, 28 ,28, 1))

# 과적합시 멈춤
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,   # 개선으로 간주되는 최소 변경 크기. 이 값만큼 개선이 없으면 Early Stopping 대상이 됩니다.
    patience=10,          # Early Stopping이 실제로 학습을 중지하기 전에 몇 epoch를 기다릴지를 의미합니다.
    restore_best_weights=True, # Early Stopping시 이전에 찾은 최적의 가중치값으로 복원합니다.
)

In [9]:
def mnist(learning_late, epochs, batch_size, dense_unit):
    model = models.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                      padding='same', activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(padding='same'),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(dense_unit, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_late), loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=epochs, verbose = 1, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=[early_stopping])

    if len(hist.epoch) < epochs:
        return 0

    return model.evaluate(X_test, y_test)[1]

In [10]:
mnist(0.005, 15, 1000, 512)

Epoch 1/15
60/60 [==============================] - 1s 9ms/step - loss: 0.3189 - accuracy: 0.8981 - val_loss: 0.0541 - val_accuracy: 0.9827
Epoch 2/15
60/60 [==============================] - 0s 7ms/step - loss: 0.0570 - accuracy: 0.9822 - val_loss: 0.0331 - val_accuracy: 0.9891
Epoch 3/15
60/60 [==============================] - 0s 7ms/step - loss: 0.0372 - accuracy: 0.9884 - val_loss: 0.0301 - val_accuracy: 0.9897
Epoch 4/15
60/60 [==============================] - 0s 7ms/step - loss: 0.0271 - accuracy: 0.9912 - val_loss: 0.0305 - val_accuracy: 0.9901
Epoch 5/15
60/60 [==============================] - 0s 7ms/step - loss: 0.0207 - accuracy: 0.9933 - val_loss: 0.0312 - val_accuracy: 0.9896
Epoch 6/15
60/60 [==============================] - 0s 7ms/step - loss: 0.0184 - accuracy: 0.9940 - val_loss: 0.0232 - val_accuracy: 0.9921
Epoch 7/15
60/60 [==============================] - 0s 7ms/step - loss: 0.0141 - accuracy: 0.9954 - val_loss: 0.0271 - val_accuracy: 0.9912
Epoch 8/15
60/60 [==

0.9909999966621399

In [11]:
def crossing(p1, p2):
    s = random.randint(1,len(p1)-1)
    e = random.randint(s,len(p1)-1)

    c = p1[:s] + p2[s:e] + p1[e:]
    return c

In [ ]:
def mutating(c):